# Fronius Solar API

In [2]:
import toml
import requests
import pandas as pd

conf = toml.load('../../conf.toml')

url_base = 'http://{}/solar_api'.format(conf['fronius'])
url = url_base + '/GetAPIVersion.cgi'

pd.DataFrame(requests.get(url).json(), [0]).set_index('APIVersion')

,BaseURL,CompatibilityRange
APIVersion,,
1,/solar_api/v1/,1.5-18


## Real-Time Data
- DAY_ENERGY: AC Energy generated on current day
- PAC: AC power (negative value for consuming power)
- TOTAL_ENERGY: AC Energy generated overall
- YEAR_ENERGY: AC Energy generated in current year

### Scope = System

In [3]:
url_real = url_base + '/v1/GetInverterRealtimeData.cgi'
url = url_real + '?Scope=System'

dic = requests.get(url).json()
dic['Head']

{'RequestArguments': {'DeviceClass': 'Inverter', 'Scope': 'System'},
 'Status': {'Code': 0, 'Reason': '', 'UserMessage': ''},
 'Timestamp': '2021-06-03T13:10:35+02:00'}

In [4]:
df = pd.DataFrame(dic['Body']['Data'])
df.loc['Values'] = df.loc['Values'].apply(lambda x: x['1'])
df

,DAY_ENERGY,PAC,TOTAL_ENERGY,YEAR_ENERGY
Unit,Wh,W,Wh,Wh
Values,10748,2054,6201509,2209592


### Scope = Device
#### Cumulation Inverter Data
Data is identical to "System".

In [5]:
url_real_dev = url_real + '?Scope=Device&DeviceId=1'
url = url_real_dev + '&DataCollection=CumulationInverterData'

dic = requests.get(url).json()
dic_data = dic['Body']['Data']
dic['Head'], dic_data['DeviceStatus']

({'RequestArguments': {'DataCollection': 'CumulationInverterData',
   'DeviceClass': 'Inverter',
   'DeviceId': '1',
   'Scope': 'Device'},
  'Status': {'Code': 0, 'Reason': '', 'UserMessage': ''},
  'Timestamp': '2021-06-03T13:10:36+02:00'},
 {'ErrorCode': 0,
  'LEDColor': 2,
  'LEDState': 0,
  'MgmtTimerRemainingTime': -1,
  'StateToReset': False,
  'StatusCode': 7})

In [6]:

t = dic['Head']['Timestamp']

In [7]:
df = {key: dic_data[key] for key in dic_data.keys() if key != 'DeviceStatus'}
pd.DataFrame(df)

,DAY_ENERGY,PAC,TOTAL_ENERGY,YEAR_ENERGY
Unit,Wh,W,Wh,Wh
Value,10750,2052,6201509.5,2209593.25


In [8]:
dic

{'Body': {'Data': {'DAY_ENERGY': {'Unit': 'Wh', 'Value': 10750},
   'DeviceStatus': {'ErrorCode': 0,
    'LEDColor': 2,
    'LEDState': 0,
    'MgmtTimerRemainingTime': -1,
    'StateToReset': False,
    'StatusCode': 7},
   'PAC': {'Unit': 'W', 'Value': 2052},
   'TOTAL_ENERGY': {'Unit': 'Wh', 'Value': 6201509.5},
   'YEAR_ENERGY': {'Unit': 'Wh', 'Value': 2209593.25}}},
 'Head': {'RequestArguments': {'DataCollection': 'CumulationInverterData',
   'DeviceClass': 'Inverter',
   'DeviceId': '1',
   'Scope': 'Device'},
  'Status': {'Code': 0, 'Reason': '', 'UserMessage': ''},
  'Timestamp': '2021-06-03T13:10:36+02:00'}}

#### Common Inverter Data
- FAC: AC frequency
- IAC: AC current (absolute, accumulated over all lines)
- IDC: DC current
- UAC: AC voltage
- UDC: DC voltage

In [9]:
url = url_real_dev + '&DataCollection=CommonInverterData'

dic = requests.get(url).json()
dic_data = dic['Body']['Data']
dic['Head'], dic_data['DeviceStatus']

({'RequestArguments': {'DataCollection': 'CommonInverterData',
   'DeviceClass': 'Inverter',
   'DeviceId': '1',
   'Scope': 'Device'},
  'Status': {'Code': 0, 'Reason': '', 'UserMessage': ''},
  'Timestamp': '2021-06-03T13:10:37+02:00'},
 {'ErrorCode': 0,
  'LEDColor': 2,
  'LEDState': 0,
  'MgmtTimerRemainingTime': -1,
  'StateToReset': False,
  'StatusCode': 7})

In [10]:
df = {key: dic_data[key] for key in dic_data.keys() if key != 'DeviceStatus'}
pd.DataFrame(df)

,DAY_ENERGY,FAC,IAC,IDC,PAC,TOTAL_ENERGY,UAC,UDC,YEAR_ENERGY
Unit,Wh,Hz,A,A,W,Wh,V,V,Wh
Value,10750,50,8.75,11.73,2049,6201509.5,234.3,186,2209594.5


#### Min Max Inverter Data

In [11]:
url = url_real_dev + '&DataCollection=MinMaxInverterData'

dic = requests.get(url).json()
dic['Head']

{'RequestArguments': {'DataCollection': 'MinMaxInverterData',
  'DeviceClass': 'Inverter',
  'DeviceId': '1',
  'Scope': 'Device'},
 'Status': {'Code': 0, 'Reason': '', 'UserMessage': ''},
 'Timestamp': '2021-06-03T13:10:38+02:00'}

In [12]:
pd.DataFrame(dic['Body']['Data'])

,DAY_PMAX,DAY_UACMAX,DAY_UDCMAX,TOTAL_PMAX,TOTAL_UACMAX,TOTAL_UDCMAX,YEAR_PMAX,YEAR_UACMAX,YEAR_UDCMAX
Unit,W,V,V,W,V,V,W,V,V
Value,4798,261,209.5,5216,267.6,246.1,5093,267.6,242.7


### Logger Info
Information about logging device.

In [13]:
url = url_base + '/v1/GetLoggerInfo.cgi'
# requests.get(url).json()

### Logger LED Info
Information of LEDs on the device.

In [14]:
url = url_base + '/v1/GetLoggerLEDInfo.cgi'
dic = requests.get(url).json()
dic['Head']

{'RequestArguments': {},
 'Status': {'Code': 0, 'Reason': '', 'UserMessage': ''},
 'Timestamp': '2021-06-03T13:10:39+02:00'}

In [15]:
pd.DataFrame(dic['Body']['Data'])

,PowerLED,SolarNetLED,SolarWebLED,WLANLED
Color,green,green,green,green
State,on,on,on,on


### Inverter Info
Information about all inverters who were online in the last 24 hours.

In [16]:
url = url_base + '/v1/GetInverterInfo.cgi'
# requests.get(url).json()

### Active Device Info
Information about online decives.

In [17]:
url = url_base + '/v1/GetActiveDeviceInfo.cgi?DeviceClass=System'
# requests.get(url).json()

### Power Flow Realtime Data
Information about the local energy grid.

In [18]:
url = url_base + '/v1/GetPowerFlowRealtimeData.fcgi'
dic = requests.get(url).json()
dic_dat = dic['Body']['Data']
dic['Head'] # , {key: dic_dat[key] for key in dic_dat.keys() if key != 'Inverters'}

{'RequestArguments': {},
 'Status': {'Code': 0, 'Reason': '', 'UserMessage': ''},
 'Timestamp': '2021-06-03T13:10:40+02:00'}

In [19]:
pd.DataFrame(dic_dat['Inverters'])

,1
DT,76.0
E_Day,10750.0
E_Total,6201509.5
E_Year,2209594.5
P,2049.0


## Archive Data
- Requests can be made for up to 16 days.
- 5 minute frequency

In [20]:
start = pd.Timestamp('2021-01-01')
end = start + pd.Timedelta('16d')
url = url_base + '/v1/GetArchiveData.cgi' \
                 '?Scope=System&StartDate={}&EndDate={}'.format(start, end)
channels = [
    'TimeSpanInSec', 'EnergyReal_WAC_Sum_Produced', 'InverterErrors',
    'Current_DC_String_1', 'Current_DC_String_2', 'Voltage_DC_String_1',
    'Voltage_DC_String_2', 'Temperature_Powerstage', 'Voltage_AC_Phase_1',
    'Current_AC_Phase_1', 'PowerReal_PAC_Sum'
]

for cha in channels:
    url += '&Channel={}'.format(cha)

dic = requests.get(url).json()

inv = dic['Body']['Data']['inverter/1']
dat = inv['Data']
dat_0 = dat[list(dat.keys())[0]]

[dic['Head'], {key: inv[key] for key in inv.keys() if key != 'Data'},
 {key_out: {key_in: dat_0[key_in] for key_in in dat_0.keys()
            if key_in != 'Values'} for key_out in dat.keys()}]

[{'RequestArguments': {'Channel': ['TimeSpanInSec',
    'EnergyReal_WAC_Sum_Produced',
    'InverterErrors',
    'Current_DC_String_1',
    'Current_DC_String_2',
    'Voltage_DC_String_1',
    'Voltage_DC_String_2',
    'Temperature_Powerstage',
    'Voltage_AC_Phase_1',
    'Current_AC_Phase_1',
    'PowerReal_PAC_Sum'],
   'EndDate': '2021-01-17T23:59:59+01:00',
   'HumanReadable': 'True',
   'Scope': 'System',
   'SeriesType': 'Detail',
   'StartDate': '2021-01-01T00:00:00+01:00'},
  'Status': {'Code': 0,
   'ErrorDetail': {'Nodes': []},
   'Reason': '',
   'UserMessage': ''},
  'Timestamp': '2021-06-03T13:10:57+02:00'},
 {'DeviceType': 76,
  'End': '2021-01-17T23:59:59+01:00',
  'NodeType': 97,
  'Start': '2021-01-01T00:00:00+01:00'},
 {'Current_AC_Phase_1': {'Unit': 'A', '_comment': 'channelId=65794'},
  'Current_DC_String_1': {'Unit': 'A', '_comment': 'channelId=65794'},
  'Current_DC_String_2': {'Unit': 'A', '_comment': 'channelId=65794'},
  'EnergyReal_WAC_Sum_Produced': {'Uni

In [21]:
df = pd.DataFrame({key: dat[key]['Values'] for key in dat.keys()})

origin = (start - pd.Timestamp('1970-01-01')) // pd.Timedelta('1 s')
dt = df.index.astype('int') + origin
df.index = pd.to_datetime(dt, unit='s').sort_values()

df


,Current_AC_Phase_1,Current_DC_String_1,Current_DC_String_2,EnergyReal_WAC_Sum_Produced,InverterErrors,PowerReal_PAC_Sum,Temperature_Powerstage,TimeSpanInSec,Voltage_AC_Phase_1,Voltage_DC_String_1,Voltage_DC_String_2
2021-01-01 08:10:00,5.00,2.02,4.34,96.588333,NaN,1170.767677,40.0,297,233.5,204.5,191.5
2021-01-01 08:15:00,8.74,2.72,8.69,172.974722,NaN,2061.950331,40.0,302,235.4,191.8,189.7
2021-01-01 08:20:00,8.08,2.26,7.91,160.368611,NaN,1911.678808,41.0,302,236.0,202.8,196.9
2021-01-01 08:25:00,6.46,1.93,6.82,125.732222,NaN,1524.026936,41.0,297,235.5,202.9,179.6
2021-01-01 08:30:00,5.61,2.22,5.16,110.955833,NaN,1322.652318,40.0,302,235.5,211.7,180.1
...,...,...,...,...,...,...,...,...,...,...,...
2021-01-17 16:40:00,3.98,2.47,2.49,78.125833,NaN,931.301325,40.0,302,233.6,199.3,197.9
2021-01-17 16:45:00,2.14,1.38,1.38,40.990000,NaN,496.848485,38.0,297,231.9,195.6,194.9
2021-01-17 16:50:00,1.22,0.83,0.84,23.760278,NaN,283.235099,37.0,302,232.0,192.2,190.8
2021-01-17 16:55:00,2.99,1.65,2.10,57.606111,NaN,698.255892,37.0,297,232.8,201.9,193.7
